# Applied Data Science Specialization

## Applied Data Science Capstone Project Notebook

### Segmenting and Clustering Neighbourhoods in Toronto

#### Created by: Xavier Jácome Piñeiros

First of all, it is necessary to import all the libraries to do the data analysis, data visualization and the unsupervised machine learning method which is K-Means clustering.

In [58]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Download and Explore Dataset

Create the dataframe with the Wikipedia data. It has been selected just the first table which contains the Postcode, Borough and Neighbourhood of London.

In [59]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0][pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Rename the dataframe df column Postcode as PostalCode.

In [60]:
df.rename(columns={df.columns[0]: "PostalCode"},inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Data Cleaning

Combine into one row the neighbourhoods that refers to the same Borough. Use comma to separate.

In [61]:
df1 = df.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join).reset_index()

In [62]:
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Put the Borough name in the Neighbourhood that is marked with 'Not assigned'

In [63]:
df1.loc[df1['Neighbourhood'] =='Not assigned', 'Neighbourhood'] = df1['Borough']
df1.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Number of rows and columns in the dataframe

In [64]:
df1.shape

(103, 3)

Let's assing the latitude and longitude to the dataframe.

In [65]:
import geocoder
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [66]:
postal_codes = df1['PostalCode']    
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df1['Latitude'] = df_coords['Latitude']
df1['Longitude'] = df_coords['Longitude']

In [68]:
df1.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035
5,M9A,Etobicoke,Islington Avenue,43.662242,-79.528379
6,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749195,-79.361905
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180


An example to show that the method has been applied.

In [69]:
df1[df1.PostalCode == 'M5G']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493


Use geopy library to get the latitude and longitude values of Toronto City

In [70]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent Toronto, as shown below.

In [71]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighbourhoods superimposed on top.

In [72]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lng, borough, Neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighbourhoods which Borough contains the word Toronto.

In [75]:
df1= df1[df1['Borough'].str.contains("Toronto")].reset_index()

In [76]:
df1.head(10)

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
1,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180
2,15,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
3,19,M4E,East Toronto,The Beaches,43.676845,-79.295225
4,20,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675
5,24,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.384930
6,25,M6G,Downtown Toronto,Christie,43.668781,-79.420710
7,30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.649515,-79.382503
8,31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.665087,-79.438705
9,36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.623470,-79.391507


In [78]:
df1.shape

(38, 6)

As we did with all of Boroughs related to Toronto, let's visualizat the neighbourhoods in it.

In [77]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lng, borough, Neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [79]:
CLIENT_ID = 'AV5N1JA1Z0PQAW2UFIDQLSYWNUH0UN1JK43GZPBBTEJQI1FD' # your Foursquare ID
CLIENT_SECRET = 'DDF1JKCLC5DQSOZKDQVDS0I2VT40KVY4TWA24RDC1EEPXJXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AV5N1JA1Z0PQAW2UFIDQLSYWNUH0UN1JK43GZPBBTEJQI1FD
CLIENT_SECRET:DDF1JKCLC5DQSOZKDQVDS0I2VT40KVY4TWA24RDC1EEPXJXX


Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [82]:
df1.loc[0, 'Neighbourhood']

'Harbourfront, Regent Park'

Get the neighborhood's latitude and longitude values.

In [85]:
neighbourhood_latitude = df1.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df1.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df1.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.65512000000007, -79.36263979699999.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [87]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AV5N1JA1Z0PQAW2UFIDQLSYWNUH0UN1JK43GZPBBTEJQI1FD&client_secret=DDF1JKCLC5DQSOZKDQVDS0I2VT40KVY4TWA24RDC1EEPXJXX&v=20180605&ll=43.65512000000007,-79.36263979699999&radius=500&limit=100'

Send the GET request and examine the resutls.

In [89]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7fec870d2be7002ce492a2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.65962000450007,
    'lng': -79.35643170823275},
   'sw': {'lat': 43.650619995500065, 'lng': -79.36884788576722}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label'

Let's borrow the get_category_type function from the Foursquare lab.

In [90]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [91]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Cocina Economica,Mexican Restaurant,43.654959,-79.365657


And how many venues were returned by Foursquare?

In [92]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


### Explore Neighbourhoods

Let's create a function to repeat the same process to all the neighbourhoods in Boroughs with the word Toronto.

In [98]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [99]:
# type your answer here

toronto_venues = getNearbyVenues(names=df1['Neighbourhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

Let's check the size of the resulting dataframe.

In [100]:
print(toronto_venues.shape)
toronto_venues.head()

(1739, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Harbourfront, Regent Park",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65512,-79.36264,Cocina Economica,43.654959,-79.365657,Mexican Restaurant


Let's check how many venues were returned for each neighborhood.

In [101]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,62,62,62,62,62,62
"Brockton, Exhibition Place, Parkdale Village",66,66,66,66,66,66
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",71,71,71,71,71,71
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,98,98,98,98,98,98
"Chinatown, Grange Park, Kensington Market",95,95,95,95,95,95
Christie,10,10,10,10,10,10


Let's find out how many unique categories can be curated from all the returned venues.

In [103]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 212 uniques categories.


### Analyze Each Neighbourhood

Analyze Each Neighborhood.

In [104]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [105]:
toronto_onehot.shape

(1739, 213)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [106]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.000000,0.060000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.090000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.030000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00

Let's confirm the new size.

In [107]:
toronto_grouped.shape

(38, 213)

Let's print each neighborhood along with the top 5 most common venues.

In [108]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          venue  freq
0   Coffee Shop  0.09
1          Café  0.06
2         Hotel  0.04
3    Restaurant  0.03
4  Burger Joint  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.05
2      Restaurant  0.05
3          Lounge  0.03
4  Farmers Market  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop  0.11
1            Café  0.06
2          Bakery  0.05
3  Sandwich Place  0.05
4      Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2        Hotel  0.04
3          Bar  0.04
4   Steakhouse  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.07
2                Park  0.04
3          Restaurant  0.04
4                Café  0.04

First, let's write a function to sort the venues in descending order and putting them in a dataframe.

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [127]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Asian Restaurant,Restaurant,American Restaurant,Gym,Seafood Restaurant,Gastropub,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Beer Bar,Cheese Shop,Lounge,Hotel,Seafood Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Sandwich Place,Bakery,Hotel,Vegetarian / Vegan Restaurant,Beer Bar,Supermarket
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Steakhouse,Hotel,Bar,Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Park,Bar,Café,Restaurant,Gym / Fitness Center,Sandwich Place,Speakeasy,Pub


### Cluster Neighbourhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [129]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1,
       4, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [130]:
toronto_grouped_clustering.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0.0,0.03,0.0,0.010000,0.0,0.000000,0.030000,0.0,0.000000,0.0,0.000000,0.020000,0.0,0.030000,0.000000,0.010000,0.0,0.000000,0.000000,0.02,0.0,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.01,0.0,0.0,0.060000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.090000,0.0,0.000000,0.0,0.01,0.000000,0.0,0.020000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.02,0.01,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.01,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.01,0.03,0.0,0.0,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.040000,0.000000,0.0,0.010000,0.00,0.0,0.000000,0.000000,0.010000,0.030000,0.000000,0.0,0.0,0.00,0.0,0.0,0.010000,0.000000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.01,0.00,0.000000,0.0,0.0,0.01,0.010000,0.000000,0.01,0.000000,0.01,0.0,0.0,0.0,0.000000,0.0000

In [131]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640,0,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Yoga Studio,Bakery,Mexican Restaurant,Food Truck,Spa,Restaurant,Event Space
1,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Tea Room,Sandwich Place,Fast Food Restaurant,Ramen Restaurant,Bookstore
2,15,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,0,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Bakery,Breakfast Spot,Gastropub,Clothing Store,Cocktail Bar
3,19,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
4,20,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675,0,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Beer Bar,Cheese Shop,Lounge,Hotel,Seafood Restaurant


In [136]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.362640,0,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Yoga Studio,Bakery,Mexican Restaurant,Food Truck,Spa,Restaurant,Event Space
1,M5B,-79.378180,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Tea Room,Sandwich Place,Fast Food Restaurant,Ramen Restaurant,Bookstore
2,M5C,-79.375481,0,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Bakery,Breakfast Spot,Gastropub,Clothing Store,Cocktail Bar
3,M4E,-79.295225,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
4,M5E,-79.373675,0,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Beer Bar,Cheese Shop,Lounge,Hotel,Seafood Restaurant
5,M5G,-79.384930,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Plaza,Tea Room,Sushi Restaurant,Sandwich Place,Fast Food Restaurant,Bookstore,Gym / Fitness Center
6,M6G,-79.420710,0,Café,Grocery Store,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Playground,Flower Shop,Flea Market,Farm
7,M5H,-79.382503,0,Coffee Shop,Café,Hotel,Asian Restaurant,Restaurant,American Restaurant,Gym,Seafood Restaurant,Gastropub,Burger Joint
8,M6H,-79.438705,0,Park,Furniture / Home Store,Gym,Bar,Bank,Bakery,Middle Eastern Restaurant,Supermarket,Bus Line,Gym / Fitness Center
10,M6J,-79.417742,0,Bar,Restaurant,Asian Restaurant,Cocktail Bar,Coffee Shop,Pizza Place,Vietnamese Restaurant,French Restaurant,Boutique,Korean Restaurant


#### Cluster 2

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M5P,-79.414405,1,Park,Construction & Landscaping,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
21,M6P,-79.462874,1,Park,Convenience Store,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
28,M4T,-79.383561,1,Park,Convenience Store,Gym,Tennis Court,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market


#### Cluster 3

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M5N,-79.41912,2,Home Service,Yoga Studio,Event Space,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


#### Cluster 4

In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5J,-79.391507,3,Harbor / Marina,Café,Music Venue,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


#### Cluster 5

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M4N,-79.387085,4,Bus Line,Swim School,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
